In [0]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

!pip install pytorch_transformers

In [0]:
import pandas as pd
import numpy as np
import unicodedata
import torch
import nltk
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

from bs4 import BeautifulSoup
from pytorch_transformers import BertTokenizer, BertModel

import re

df = pd.read_csv('/content/gdrive/My Drive/IMDB_Dataset.csv', encoding='utf-8')


In [0]:
# Sentiment 1, 0으로 바꾸기
df['sentiment'] = (df['sentiment'] == 'positive').astype(int)

In [0]:
# 문장 단위로 쪼개기
df['review'] = df.review.map(sent_tokenize)
raw_reviews = df.values

In [0]:
# 클렌징
stop = stopwords.words('english')
lmtzr = WordNetLemmatizer()


def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

def clean_review(text: str) -> str:
    # 1. unicode to ASCII
    text = unicodeToAscii(text)
            
    # 2. Remove HTML
    text = BeautifulSoup(text, "lxml").get_text()
    
    # 4. Tokenize
    text = [word for word in nltk.word_tokenize(text)]
    
    # 5. Remove Stop Words
    
    text = [word for word in text if word not in stop and len(word) >= 3]
    

    return text

In [ ]:
reviews = []
  
for review in raw_reviews:
    text = review[0]
    code = review[1]
    for s in text:
        new_s = clean_review(s)
        if(len(new_s) > 10 and new_s[0].isalpha()):
            reviews.append((new_s, code))

In [0]:
import pickle
import gzip

with gzip.open('/content/gdrive/My Drive/pre_reviews.pickle', 'wb') as f:
    pickle.dump(reviews, f)